In [1]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf

objc[99318]: Class CaptureDelegate is implemented in both /Users/vipulmunot/miniforge3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x16050e538) and /Users/vipulmunot/miniforge3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x135788860). One of the two will be used. Which one is undefined.
objc[99318]: Class CVWindow is implemented in both /Users/vipulmunot/miniforge3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x16050e588) and /Users/vipulmunot/miniforge3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x117ff0a68). One of the two will be used. Which one is undefined.
objc[99318]: Class CVView is implemented in both /Users/vipulmunot/miniforge3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x16050e5b0) and /Users/vipulmunot/miniforge3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x117ff0a90). One of the two will be used. Which one is undefined.
objc[99318]: Class CVSlider is implemented in b

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results = model.process(image)
    image.flags.writeable=True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [5]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    ret, frame = cap.read()
    image, results = mediapipe_detection(frame, holistic)
    print(results)
    draw_landmarks(image,results)
    cv2.imshow('OpenCV Feed', image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break
  cap.release()
  cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    face = np.array([[res.x,res.y,res.z,] for res in results.face_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(1404)
    return np.concatenate([pose,face,lh,rh])

In [7]:
result_test = extract_keypoints(results)

In [8]:
np.save('0',result_test)

In [9]:
np.load('0.npy')

array([ 0.51195627,  0.44541439, -0.63765705, ...,  0.31152731,
        0.36943471, -0.01497335])

In [10]:
DATA_PATH = os.path.join('MP_Data')
actions = np.array(['hello','fuckyou', 'thankyou'])
no_sequences = 20
sequence_length = 20

In [11]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

In [12]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  for action in actions:
    for sequence in range(no_sequences):
        for frame_num in range(sequence_length):
            ret, frame = cap.read()
            image, results = mediapipe_detection(frame, holistic)
            print(results)
            draw_landmarks(image,results)
            if frame_num == 0:
                cv2.putText(image,'STARTING COLLECTION',(120,200), cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),1, cv2.LINE_AA)
                cv2.putText(image,'Collecting frames for {} Video number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1, cv2.LINE_AA)
                cv2.waitKey(2000)
            else:
                cv2.putText(image,'Collecting frames for {} Video number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1, cv2.LINE_AA)
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
            np.save(npy_path,keypoints)
            cv2.imshow('OpenCV Feed', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [13]:
cap.release()
cv2.destroyAllWindows()

In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 

In [15]:
label_map = {label:num for num, label in enumerate(actions)}

In [16]:
label_map

{'hello': 0, 'fuckyou': 1, 'thankyou': 2}

In [83]:
sequences, labels = [],[]
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            res = res.astype('float32')
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [64]:
np.array(sequences).shape

/var/folders/sk/d6j_zpb574909_03hgc3gy400000gn/T/ipykernel_99318/1188987122.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(sequences).shape


(60, 20)

In [19]:
np.array(labels).shape

(60,)

In [104]:
#X = np.array(sequences)
X = np.asarray(sequences).astype('float32')

/var/folders/sk/d6j_zpb574909_03hgc3gy400000gn/T/ipykernel_99318/2997050455.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(sequences).astype('float32')


ValueError: setting an array element with a sequence.

In [85]:
X.shape

(60, 20)

In [89]:
#y = to_categorical(labels).astype('object')
y = to_categorical(labels).astype('float32')

In [90]:
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.05)

In [92]:
X_train.shape

(57, 20)

In [93]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [94]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [95]:
X.shape

(60, 20)

In [173]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(20,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))

In [174]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=[tf.keras.metrics.categorical_accuracy])

In [175]:
print(tb_callback)

In [176]:
y_train

<tf.Tensor: shape=(57, 3), dtype=float32, numpy=
array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
      

In [179]:
#X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
#X_train = tf.ragged.constant(X_train)

In [103]:
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)

In [180]:
model.fit(X_train, y_train,epochs=180, callbacks=[tb_callback])

Epoch 1/180


2022-07-20 22:21:23.995859: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 7s 2s/step - loss: 2.5779 - categorical_accuracy: 0.3509
Epoch 2/180
2/2 [==============================] - 3s 2s/step - loss: 1.2007 - categorical_accuracy: 0.3509
Epoch 3/180
2/2 [==============================] - 3s 2s/step - loss: 1.4883 - categorical_accuracy: 0.3158
Epoch 4/180
2/2 [==============================] - 3s 2s/step - loss: 1.1079 - categorical_accuracy: 0.3509
Epoch 5/180
2/2 [==============================] - 3s 2s/step - loss: 1.1429 - categorical_accuracy: 0.3333
Epoch 6/180
2/2 [==============================] - 3s 2s/step - loss: 1.3101 - categorical_accuracy: 0.4561
Epoch 7/180
2/2 [==============================] - 3s 2s/step - loss: 1.3312 - categorical_accuracy: 0.3158
Epoch 8/180
2/2 [==============================] - 3s 2s/step - loss: 1.1708 - categorical_accuracy: 0.2807
Epoch 9/180
2/2 [==============================] - 3s 2s/step - loss: 1.1169 - categorical_accuracy: 0.3684
Epoch 10/180
2/2 [======================

In [181]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 20, 64)            442112    
                                                                 
 lstm_19 (LSTM)              (None, 20, 128)           98816     
                                                                 
 lstm_20 (LSTM)              (None, 64)                49408     
                                                                 
 dense_18 (Dense)            (None, 64)                4160      
                                                                 
 dense_19 (Dense)            (None, 32)                2080      
                                                                 
 dense_20 (Dense)            (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

In [182]:
#X_test = tf.ragged.constant(X_test)

In [185]:
res = model.predict(X_test)

1/1 [==============================] - 0s 143ms/step


In [186]:
actions[np.argmax(res[0])]

'fuckyou'

In [187]:
actions[np.argmax(y_test[0])]

'fuckyou'

In [188]:
model.save('action.h5')

In [189]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [191]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 159ms/step


In [192]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()

In [193]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[2, 0],
        [0, 1]],

       [[1, 0],
        [0, 2]]])

In [194]:
accuracy_score(ytrue,yhat)

1.0

In [199]:
sequence = []
sentence = []
threshold = 0.4
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image,results)

        keypoints = extract_keypoints(results).astype('float32')
        sequence.append(keypoints)
        sequence = sequence[-20:]
        if len(sequence) == 20:
            res = model.predict(np.expand_dims(sequence,axis=0))[0]
            cv2.putText(image,actions[np.argmax(res)], (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
             
        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

Results are <class 'mediapipe.python.solution_base.SolutionOutputs'>
Results are <class 'mediapipe.python.solution_base.SolutionOutputs'>
Results are <class 'mediapipe.python.solution_base.SolutionOutputs'>
Results are <class 'mediapipe.python.solution_base.SolutionOutputs'>
Results are <class 'mediapipe.python.solution_base.SolutionOutputs'>
Results are <class 'mediapipe.python.solution_base.SolutionOutputs'>
Results are <class 'mediapipe.python.solution_base.SolutionOutputs'>
Results are <class 'mediapipe.python.solution_base.SolutionOutputs'>
Results are <class 'mediapipe.python.solution_base.SolutionOutputs'>
Results are <class 'mediapipe.python.solution_base.SolutionOutputs'>
Results are <class 'mediapipe.python.solution_base.SolutionOutputs'>
Results are <class 'mediapipe.python.solution_base.SolutionOutputs'>
Results are <class 'mediapipe.python.solution_base.SolutionOutputs'>
Results are <class 'mediapipe.python.solution_base.SolutionOutputs'>
Results are <class 'mediapipe.pyth

In [200]:
cap.release()
cv2.destroyAllWindows()